<a href="https://colab.research.google.com/github/mjavadzadeh11/Crawler/blob/master/GoogleNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GoogleNews
!pip install newspaper3k

In [ ]:
!pip install googleanalytics

In [ ]:
import pandas as pd
from GoogleNews import GoogleNews
from newspaper import Article
import requests
import nltk
nltk.download('punkt')

In [ ]:
googlenews = GoogleNews(start='11/04/2021',end='12/04/2021')
googlenews.search('Iran')

#googlenews = GoogleNews(period='7d')
googlenews.get_page(10)
Result=googlenews.results()

In [ ]:
df=pd.DataFrame(Result)
df.drop(columns=['img'], inplace=True)

In [ ]:
#filtering by media
filt = df['media']== 'Al Jazeera'
df[filt]

In [ ]:
df.iloc[0,0]

In [ ]:
#grouping by media
media_gp= df.groupby(['media'])
media_gp ['link'].value_counts() 

In [ ]:

for index in range(df.shape[0]):
  try:
    article=Article(df.loc[index,'link'])
    article.download()
    article.html
    article.parse()
    article.nlp()
    
    
    df.loc[index,'Context'] = article.text
    df.loc[index, 'Summary'] = article.summary
    df.loc[index, 'Keywords'] = ' '.join(article.keywords)
  except:
    df.loc[index,'Context'] = 'Not able to recieve this news'
    df.loc[index, 'Summary'] = 'Could not summarize the context'
    df.loc[index, 'Keywords'] = "No keywords founded"
    

#df['Context'] = news_context




In [ ]:
df

In [ ]:
!pip install vaderSentiment

In [ ]:
# sentiment analysis     # !pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [ ]:
Analyzer = SentimentIntensityAnalyzer()
negative=[]
netrual=[]
positive=[]
for i in range(df.shape[0]):
  title = df.iloc[i,0]
  desc	 = df.iloc[i,4]
  summ = df.loc[i,'Summary']
  title_analyzer = Analyzer.polarity_scores(title)
  caption_analyzer = Analyzer.polarity_scores(desc)
  summary_analyzer = Analyzer.polarity_scores(summ)
  negative.append(((title_analyzer['neg']) + (caption_analyzer['neg']) + (summary_analyzer['neg']) )/3)
  positive.append(((title_analyzer['pos']) + (caption_analyzer['pos']) + (summary_analyzer['pos']))/3)
  netrual.append(((title_analyzer['neu']) + (caption_analyzer['neu'])+ (summary_analyzer['neu']))/3)

df['Negative']= negative
df['Positive']= positive
df['Netrual'] = netrual

In [ ]:
df

In [1]:
df.nlargest(5,['Negative'])

NameError: ignored